In [65]:
import numpy as np
import pandas as pd
import re
import codecs
import requests
import progressbar
import os
import json

import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

import lucene
from org.apache.lucene.analysis.standard import StandardTokenizer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.analysis.tokenattributes import CharTermAttribute
from org.apache.lucene.util import Version
from java.io import StringReader

In [3]:
lucene.initVM()
print Version.LUCENE_CURRENT

6.2.0


In [4]:
arpabet = nltk.corpus.cmudict.dict()

base_url = 'http://www.speech.cs.cmu.edu/cgi-bin/tools/logios/lextool.pl'

file = {'wordfile': ('words.txt', "\n".join([str(t).upper() for line in tokens for t in line if t not in arpabet.keys()]))}

res = requests.post(base_url, files=file, allow_redirects=True)

dict_re = re.compile(r"(?<=DICT ).*?\.dict")

dict_url = dict_re.search(res.text).group(0)

res_dict = requests.get(dict_url)

NameError: name 'tokens' is not defined

In [5]:
def tokenize(string):
    tokenizer = StandardTokenizer()
    tokenizer.setReader(StringReader(string))
    token = tokenizer.addAttribute(CharTermAttribute.class_)
    tokenizer.reset()
    lucene_tokens = []
    while tokenizer.incrementToken():
        lucene_tokens.append(token.toString())
    return lucene_tokens

def normalize(string):
    analyzer = EnglishAnalyzer()
    tokenstream = analyzer.tokenStream('token', StringReader(string))
    token = tokenstream.addAttribute(CharTermAttribute.class_)
    tokenstream.reset()
    tokens = []
    while tokenstream.incrementToken():
        if not re.search('\d', token.toString()):
            tokens.append(token.toString())
    return tokens

# def get_phonemes(word):
#     try:
#         return arpabet[word][0]
#     except:
#         try:
#             return custom_dict[word]
#         except:
#             print(word)
#             return word
        
def get_speakers_list(speakers_string):
    speakers = speakers_string.upper()
    speakers = re.sub(' AND ', '/', speakers)
    speakers = re.sub(' & ', '/', speakers)
    speakers = re.sub('\[.*\d ', '', speakers)
    speakers = re.sub(' \(.*\)', '', speakers)
    speakers = re.sub('CHORUS ', '', speakers)
    speakers = re.sub(' EXCEPT.*\]', '', speakers)
    speakers = re.sub('ALEXANDER HAMILTON', 'HAMILTON', speakers)
    speakers = re.sub('GEORGE WASHINGTON', 'WASHINGTON', speakers)
    speakers = re.sub('AARON BURR', 'BURR', speakers)
    speakers = re.sub('JAMES MADISON', 'MADISON', speakers)
    speakers = re.sub('\[ENS\]', 'ENSEMBLE', speakers)
    speakers = re.sub('\[WASH\]', 'WASHINGTON', speakers)
    speakers = re.sub('\[WASH/', 'WASHINGTON/', speakers)
    speakers = re.sub('/MULL/', '/MULLIGAN/', speakers)
    speakers = re.sub('/LAUR/', '/LAURENS/', speakers)
    speakers = re.sub('/LAF\]', '/LAFAYETTE', speakers)
    speakers = re.sub('\[', '', speakers)
    speakers = re.sub('\]', '', speakers)
    speakers = speakers.strip().split('/')
    return speakers

In [74]:
hamilton_data = '../data/raw/hamilton/'
hamilton_json = []
for filename in os.listdir(hamilton_data):
    with codecs.open(hamilton_data + filename, encoding='utf-8', errors='replace') as f:
        lines = f.readlines()
    track_dict = dict()
    track_dict['act#'], track_dict['track#'], track_dict['track'] = filename[:-4].split('_')
    lyrics = []
    current_speaker = None
    i = 0
    for line in lines:
        if len(line.strip()) == 0: continue
        if re.match('^\[.*\]$', line.strip()):
            current_speaker = get_speakers_list(line)
        else:
            line_dict = {
                'line#': i,
                'speakers': current_speaker,
                'original': line.strip(),
                'tokenized': tokenize(line),
                'normalized': tokenize(line)#,
                #'phonemes': get_phonemes(line)
            }
            lyrics.append(line_dict)
            i += 1
    track_dict['lyrics'] = lyrics
    hamilton_json.append(track_dict)

In [75]:
hamilton_json

[{'act#': '1',
  'lyrics': [{'line#': 0,
    'normalized': [u'Hey', u'hey', u'hey', u'hey'],
    'original': u'Hey hey hey hey',
    'speakers': [u'HAMILTON', u'BURR', u'LAURENS', u'ALL WOMEN'],
    'tokenized': [u'Hey', u'hey', u'hey', u'hey']},
   {'line#': 1,
    'normalized': [u'Ohh', u'I', u'do', u'I', u'do', u'I', u'do', u'I'],
    'original': u'Ohh, I do I do I do I',
    'speakers': [u'ELIZA'],
    'tokenized': [u'Ohh', u'I', u'do', u'I', u'do', u'I', u'do', u'I']},
   {'line#': 2,
    'normalized': [u'Dooo', u'Hey'],
    'original': u'Dooo! Hey!',
    'speakers': [u'ELIZA'],
    'tokenized': [u'Dooo', u'Hey']},
   {'line#': 3,
    'normalized': [u'Ohh', u'I', u'do', u'I', u'do', u'I', u'do', u'I'],
    'original': u'Ohh, I do I do I do I',
    'speakers': [u'ELIZA'],
    'tokenized': [u'Ohh', u'I', u'do', u'I', u'do', u'I', u'do', u'I']},
   {'line#': 4,
    'normalized': [u'Dooo', u'Boy', u'you', u'got', u'me'],
    'original': u'Dooo! Boy you got me',
    'speakers': [u'ELIZ

In [76]:
with open('../data/processed/hamilton_data.json', 'w') as f:
    json.dump(hamilton_json, f)

In [64]:
hamilton_json[0]['lyrics'][10]['normalized']

[u'Look',
 u'into',
 u'your',
 u'eyes',
 u'and',
 u'the',
 u'sky',
 u's',
 u'the',
 u'limit',
 u'I',
 u'm',
 u'helpless']

In [66]:
sa = SentimentAnalyzer()

In [71]:
sa.

AttributeError: 'NoneType' object has no attribute 'classify_many'